In [1]:
##### Batas penggunaan model yang sudah jadi #############

In [2]:
import tensorflow as tf
from tensorflow.keras import models
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import jsons
from flask import Flask, jsonify, request

In [3]:
def transformator(file_gambar_spectogram):
    IMG_PATH = file_gambar_spectogram
    image = cv2.imread(IMG_PATH)
    image = cv2.resize(image, (512, 512))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

def identifikasi_gambar(gambar):
    y_pred = np.argmax(model.predict(transformator(gambar)), axis=1)
    if str(y_pred[0]) == "0":
       label_nama = "Depresi"
    elif str(y_pred[0]) == "1":
       label_nama = "Normal"
    return label_nama

def transform_audio_to_spectogram_to_testing(audio_file):
    y, sr = librosa.load(audio_file)
    librosa.feature.melspectrogram(y=y, sr=sr)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,
                                               fmax=8000)
    fig, ax = plt.subplots()
    S_dB = librosa.power_to_db(S, ref=np.max)
    img = librosa.display.specshow(S_dB)
            # ax.set(title='Log Mel-frequency spectrogram')
    folder_dituju = "E:\Capstone_Project\\runningan\jalur_lewat_testing" #save suatu folder diserver
    savefigure = os.path.join(folder_dituju, 'jalur_lewat.jpg') # save di folder folder_dituju , nama file gambar 'jalur_lewat.jpg' / spectogram
    plt.savefig(savefigure)
    status = identifikasi_gambar(savefigure)
    return status

AUTOTUNE = tf.data.AUTOTUNE

In [4]:
#audio_file = "E:\Capstone_Project\Depressionvsnormal\Depression\Depression4.wav"
#hasil = transform_audio_to_spectogram_to_testing(audio_file)

#print(hasil)

In [5]:
model_ml = "E:\Capstone_Project\heroku\\predictive_model_v_8.h5"
model = models.load_model(model_ml)

In [ ]:
from flask import Flask
from gevent.pywsgi import WSGIServer

app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def infer_image():
    if request.method == 'POST':
        if 'file' not in request.files:
            return "Masukan hanya jenis file saja"
        file = request.files.get('file')
        try:
            img = transform_audio_to_spectogram_to_testing(file)
            json =  { "status_user":img,"status_running":"Sukses"} 
        except:
            json =  { "status_user":"-","status_running":"Gagal"}
        return json
    
@app.route('/', methods=['GET'])
def index():
    return "Selamat datang dimachine learning kami"

if __name__ == '__main__':
    # Debug/Development
    # app.run(debug=True, host="0.0.0.0", port="5000")
    # Production
    http_server = WSGIServer(('127.0.0.1', 5000), app)
    http_server.serve_forever()

1/1 [==============================] - 0s 220ms/step


127.0.0.1 - - [2022-06-08 11:37:18] "POST /predict HTTP/1.1" 200 159 1.475208
